<a href="https://colab.research.google.com/github/BrunoGeraldine/B3_Prediction_2023/blob/main/notebook/B3_LTSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B3_Prediction_2023 → LTSM

## Bibliotecas e help functions

In [ ]:
# Import
import sidetable
import numpy                 as np
import pandas                as pd
import seaborn               as sns
import matplotlib.pyplot     as plt

from tensorflow.keras.models import Sequential
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, LSTM, Dropout


In [ ]:
!pip install sidetable

In [ ]:
## Help Functions

# Supress scientific Notation
#np.set_printoptions(suppress=True)
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Read data

In [ ]:
df = pd.read_parquet('/content/all_bovespa.parquet')

In [ ]:
df.head()

# Action_Filter

In [ ]:
df_acao = df.copy()

In [ ]:
sigla = 'PETR4'
df_acao = df[df['sigla_acao'] == sigla ]

In [ ]:
df_acao

In [ ]:
# Selecionando os campos que vamos análisar
df_acao_fech = df_acao[['data_pregao', 'preco_fech']]


In [ ]:
# Ajustando o indice pela data
df_acao_fech = df_acao_fech.set_index(pd.DatetimeIndex(df_acao_fech['data_pregao'].values))

In [ ]:
# Eliminano a coluna 'data_pregao'

df_acao_fech.drop(columns=['data_pregao'], inplace=True)

# EDA → Action

## Verificar quantidade de linhas

In [ ]:
print( f'Number of Rows: {df_acao_fech.shape[0]}' )
print( f'Number of Columns: {df_acao_fech.shape[1]}' ) 

## Data types

In [ ]:
df_acao_fech.dtypes

## Check NA

In [ ]:
df_acao_fech.stb.missing()

## Data description

In [ ]:
df_acao_fech.describe()

## Grafico dos valores de fechamento

In [ ]:
# Plotando o comportamento da ação ao longo do periodo 2019 > 2023

plt.figure(figsize=(25,8))
plt.title('Preco de fechamento 2019 à 2023')
plt.plot(df_acao_fech['preco_fech'], color='darkslateblue' )
plt.xlabel('data');


# Train_Test_Split

In [ ]:
# coletando a quantidade de linhas
qtd_linhas = len(df_acao_fech)

qtd_linhas_treino =round(.70 * qtd_linhas)

qtd_linhas_test = qtd_linhas - qtd_linhas_treino

info = (
    f'Linhas de treino = 0:{qtd_linhas_treino}, '
    f'linhas de teste = {qtd_linhas_treino}:{qtd_linhas_treino+qtd_linhas_test}'
)

info

# Data Modeling

In [ ]:
# Normalização dos dados

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_acao_fech)

## Select Train_data and Test_data

In [ ]:
train = df_scaled[:qtd_linhas_treino]
test = df_scaled[qtd_linhas_treino:qtd_linhas_treino+qtd_linhas_test]

print(len(train), len(test))

In [ ]:
# Função que trasnforma os dados no tipo de dados que o LTSM requisita para o funcionamento
def creat_df(df, steps=1):
  dataX, dataY = [], []
  for i in range(len(df)-steps-1):
    a = df[i:(i+steps), 0]
    dataX.append(a)
    dataY.append(df[i + steps, 0])
  
  return np.array(dataX), np.array(dataY)

""" Formata os valores que a rede LSTM espera """

In [ ]:
# Gerando os dados de treino e test
steps = 15 #(dias a prever)
X_train, Y_train = creat_df(train, steps)
X_test, Y_test = creat_df(test, steps)


In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
# Gerando os dados que o modelo espera

# O numero 1 no final do codigo refere-se a quantidade de features explicativos que o estamos passando
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
# Montando a rede LSTM
model = Sequential()
model.add(LSTM(35, return_sequences=True, input_shape=(steps, 1)))
# return_sequences=True → retorna a informação selecionada acima e reaplica o modelo 
model.add(LSTM(35, return_sequences=True))
model.add(LSTM(35))
model.add(Dropout(0.2)) #→ organiza as features para evitar overfit
model.add(Dense(1)) #→saida do modelo, no caso 1 (unica feature preco) "resposta preco previsto"

In [ ]:
# Compilando os dados da saida
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
# Treinamento do Modelo
validation = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, batch_size=15, verbose=2)

In [ ]:
# grafico do comportamento do modelo
plt.figure(figsize=(20,8))
plt.plot(validation.history['loss'], label='Training loss')
plt.plot(validation.history['val_loss'], label='Validation loss')
plt.legend();

In [ ]:
# Fazendo a previsão
prev = model.predict(X_test)
prev = scaler.inverse_transform(prev)
prev

In [ ]:
# Prevendo os proximos 10 dias
lenght_test = len(test)
lenght_test

In [ ]:
# Pegar os ultimos dias que são o tamanho do steps declarado anteriormente
days_input_steps = lenght_test - steps
days_input_steps

In [ ]:
#Transforma em array
input_steps = test[days_input_steps:]
input_steps = np.array(input_steps).reshape(1, -1)
input_steps

In [ ]:
# Transfoprmar um lista
list_output_steps = list(input_steps)
list_output_steps = list_output_steps[0].tolist()
list_output_steps

In [ ]:
# Loop para prever os proximos 10 dias
#Loop to preview the next 10 days
pred_output = [] #→ recebe os dias a serem previstos
i = 0
n_future = 10
while(i<n_future):
  
    if(len(list_output_steps) > steps):
       input_steps = np.array(list_output_steps[1:])
       print('Day {}. Value Entrance → {}'.format(i, input_steps))
       input_steps = input_steps.reshape(1, -1)
       input_steps = input_steps.reshape((1, steps, 1))
       #print(input_steps)
       pred = model.predict(input_steps, verbose=0)
       print('Day {}. Value Predict → {}'.format(i, pred))
       list_output_steps.extend(pred[0].tolist())
       list_output_steps = list_output_steps[1:]
       #print(list_output_steps)
       pred_output.extend(pred.tolist())
       i = i+1
       
    else:
       input_steps = input_steps.reshape((1, steps, 1))
       pred= model.predict(input_steps, verbose=0)
       print(pred[0])
       list_output_steps.extend(pred[0].tolist())
       print(len(list_output_steps))
       pred_output.extend(pred.tolist())
       i = i+1
       
print(pred_output)
       
       


In [ ]:
# transformando o output  para lermos 
prev = scaler.inverse_transform(pred_output)
prev = np.array(prev).reshape(1,-1)
list_output_prev = list(prev)
list_output_prev = prev[0].tolist()
list_output_prev



In [ ]:
# Colhendo as datas de previsão
dates = pd.to_datetime(df_acao['data_pregao'])
predict_dates = pd.date_range(list(dates)[-1] + pd.DateOffset(1), periods=10, freq='b').tolist()
predict_dates


In [ ]:
# Criando o DataFrame de previsão
forecast_dates = []
for i in predict_dates:
  forecast_dates.append(i.date())

df_forecast = pd.DataFrame({'data_pregao': np.array(forecast_dates), 'preco_fechamento': list_output_prev})
df_forecast['data_pregao'] = pd.to_datetime(df_forecast['data_pregao'])

# transformando o index pela data
df_forecast = df_forecast.set_index(pd.DatetimeIndex(df_forecast['data_pregao'].values))
df_forecast.drop(columns='data_pregao', inplace=True)
df_forecast

In [ ]:
# Filtrando apenas os dias utilizados para previsão
df_acao = df[(df['sigla_acao'] == sigla) & (df['data_pregao'] > '2023-04-01') ]
df_acao_fech = df_acao[['data_pregao', 'preco_fech']]
df_acao_fech = df_acao_fech.set_index(pd.DatetimeIndex(df_acao_fech['data_pregao'].values))
df_acao_fech.drop(columns='data_pregao', inplace=True)

In [ ]:
#Plotar o grafico
plt.figure(figsize=(16,8))
plt.title('Price Share Value')
plt.plot(df_acao_fech['preco_fech'])
plt.plot(df_forecast['preco_fechamento'])
plt.legend(['Price Closed', 'Price Predicted'])
plt.show();

In [ ]:
# Fazer um novo ciclo acrescentando o ano de 2018